<a href="https://colab.research.google.com/github/bytehub-ai/code-examples/blob/main/tutorials/04_using_cloud_storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using cloud storage buckets

ByteHub is built on [Dask](https://docs.dask.org/en/latest/), which includes utilities for saving/loading data onto cloud-based storage services such as [AWS S3](https://aws.amazon.com/s3/), [Azure Blob](https://azure.microsoft.com/en-gb/services/storage/blobs/), [GCP Cloud Storage](https://cloud.google.com/storage).

## AWS

This tutorial demonstrates how to save feature data onto AWS S3. Start by installing ByteHub with the `aws` option.

In [1]:
!pip install -q bytehub[aws]

In [2]:
import pandas as pd
import numpy as np
import os
import shutil
import bytehub as bh
print(f'ByteHub version {bh.__version__}')

ByteHub version 0.3.3


In [3]:
# Remove any previously created feature stores
try:
    os.remove('bytehub.db')
except FileNotFoundError:
    pass
try:
    shutil.rmtree('/tmp/featurestore/tutorial')
except FileNotFoundError:
    pass

Create a new featurestore - this will be stored in a local sqlite database named `bytehub.db`.

In [4]:
fs = bh.FeatureStore()

Next we need to create a namespace within the feature store that will allow us to save data to S3. [Follow this guide](https://medium.com/@shamnad.p.s/how-to-create-an-s3-bucket-and-aws-access-key-id-and-secret-access-key-for-accessing-it-5653b6e54337) to create an S3 storage bucket and access keys, then configure them in the cell below.

In [5]:
#@title Configure S3 bucket
bucket_name = "bytehub-demo" #@param {type:"string"}

In [ ]:
from getpass import getpass
print('Input AWS access key ID:')
aws_access_key_id = getpass()
print('Input AWS secret access key:')
aws_secret_access_key = getpass()

Input AWS access key ID:
··········
Input AWS secret access key:
··········


In [ ]:
# Create the namespace on the AWS storage bucket
fs.create_namespace(
    's3-demo',
    url=f's3://{bucket_name}/demo',
    description='S3 tutorial',
    storage_options={
        'key': aws_access_key_id, 'secret': aws_secret_access_key, 'use_ssl': True
    }
)

For details on how to configure for other cloud platforms see [here](https://docs.dask.org/en/latest/remote-data-services.html#amazon-s3).

Now create a feature in this namespace.

In [ ]:
fs.create_feature('s3-demo/numbers', description='Timeseries of random numbers')

Now we can generate a Pandas dataframe with time and value columns to store.

In [ ]:
dts = pd.date_range('2020-01-01', '2021-02-09')
df = pd.DataFrame({'time': dts, 'value': np.random.randn(len(dts))})

df.head()

,time,value
0,2020-01-01,0.491179
1,2020-01-02,1.282287
2,2020-01-03,-0.719213
3,2020-01-04,1.268211
4,2020-01-05,1.309990


In [ ]:
fs.save_dataframe(df, 's3-demo/numbers')

This data is now stored in the cloud storage bucket. The underlying files can be viewed in the [AWS console](https://s3.console.aws.amazon.com/s3/home).

In [ ]:
# Query the data
result = fs.load_dataframe('s3-demo/numbers')
result.head()

,s3-demo/numbers
time,
2020-01-01,0.491179
2020-01-02,1.282287
2020-01-03,-0.719213
2020-01-04,1.268211
2020-01-05,1.309990


In [ ]:
# Delete the data saved in the tutorial
fs.delete_feature('s3-demo/numbers', delete_data=True) # delete_data will remove the data from S3

## GCP

In this section we'll demonstrate Google Cloud storage. Install the required dependencies using the `gcp` option.

In [5]:
!pip install -q bytehub[gcp]

Now setup a namespace on a Google Cloud storage bucket. Authenticate using the interative login below, or alternatively you will to go to the IAM console, create a [service account](https://console.cloud.google.com/iam-admin/serviceaccounts/) with permissions to use the storage bucket, then download keys for this account in JSON format. Supply the credentials to the `token` field in `storage_options`.

In [6]:
#@title Configure S3 bucket and project name
bucket_name = "bytehub-demo" #@param {type:"string"}
project_name = "bytehub" #@param {type:"string"}

In [8]:
def get_gcsfs(project_name):
    import gcsfs
    import google
    import json

    google.colab.auth.authenticate_user()
    creds, project = google.auth.default()

    gcs = gcsfs.GCSFileSystem(token=creds, project=project_name)

    return json.loads(gcs.credentials.to_json())

Authenticate to GCP using the cell below - we'll use the credentials when setting up the namespace.

In [9]:
token = get_gcsfs(project_name)

In [10]:
# Create the namespace on the GCP storage bucket
fs.create_namespace(
    'gcp-demo',
    url=f'gs://{bucket_name}/demo',
    description='GCP tutorial',
    storage_options={
        # Alternatively `token` could be the contents of the JSON key file
        'token': token
    }
)

Now create a feature in this namespace and insert a dataframe

In [11]:
fs.create_feature('gcp-demo/numbers', description='Timeseries of random numbers')

In [12]:
dts = pd.date_range('2020-01-01', '2020-02-09')
df = pd.DataFrame({'time': dts, 'value': np.random.randn(len(dts))})

df.head()

,time,value
0,2020-01-01,-1.511410
1,2020-01-02,-0.618672
2,2020-01-03,-0.618044
3,2020-01-04,-1.990424
4,2020-01-05,-0.388601


In [13]:
fs.save_dataframe(df, 'gcp-demo/numbers')

The data is now stored on GCP. Read it back as follows:

In [14]:
# Query the data
result = fs.load_dataframe('gcp-demo/numbers')
result.head()

,gcp-demo/numbers
time,
2020-01-01,-1.511410
2020-01-02,-0.618672
2020-01-03,-0.618044
2020-01-04,-1.990424
2020-01-05,-0.388601
